In [ ]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Writing reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [21]:
%%bash

OUT_DIR=wordcount_result
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

0%however	1
0&\mathrm{if	1
0(8)320-1234	1
0)).(1	2
0,03	1
0,1,...,n	1
0,1,0	1
0,1,\dots,n	1
0,5	1
0,50	1


17/11/23 16:55:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 16:55:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 16:55:23 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/23 16:55:23 INFO mapreduce.JobSubmitter: number of splits:2
17/11/23 16:55:23 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1511394121436_0007
17/11/23 16:55:24 INFO impl.YarnClientImpl: Submitted application application_1511394121436_0007
17/11/23 16:55:24 INFO mapreduce.Job: The url to track the job: http://be195d52a7ef:8088/proxy/application_1511394121436_0007/
17/11/23 16:55:24 INFO mapreduce.Job: Running job: job_1511394121436_0007
17/11/23 16:55:30 INFO mapreduce.Job: Job job_1511394121436_0007 running in uber mode : false
17/11/23 16:55:30 INFO mapreduce.Job:  map 0% reduce 0%
17/11/23 16:55:46 INFO mapreduce.Job:  map 30% reduce 0%
17/11/23 16:55:52 INFO mapreduce.Job:  map 46% reduce 0%
17/11/23 16:55:58 INFO 

In [22]:
%%writefile mapper1.py

import sys

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
    except ValueError as e:
        continue      
    print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
    print "%s\t%s" % (count,key)

Overwriting mapper1.py


In [24]:
%%writefile reducer1.py

import sys

index = 0

for line in sys.stdin:
    try:
        count, key = line.strip().split('\t', 1)        
    except ValueError as e:
        continue

    index+=1
    if index == 7:        
        print "%s\t%s" % (key, count)
        

Overwriting reducer1.py


In [25]:
%%bash

IN_DIR=wordcount_result
OUT_DIR=wordsrating_result
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options=-nr \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input ${IN_DIR} \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

is	126420


17/11/23 16:58:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 16:58:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 16:58:39 INFO mapred.FileInputFormat: Total input files to process : 8
17/11/23 16:58:39 INFO mapreduce.JobSubmitter: number of splits:8
17/11/23 16:58:39 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1511394121436_0009
17/11/23 16:58:39 INFO impl.YarnClientImpl: Submitted application application_1511394121436_0009
17/11/23 16:58:39 INFO mapreduce.Job: The url to track the job: http://be195d52a7ef:8088/proxy/application_1511394121436_0009/
17/11/23 16:58:39 INFO mapreduce.Job: Running job: job_1511394121436_0009
17/11/23 16:58:45 INFO mapreduce.Job: Job job_1511394121436_0009 running in uber mode : false
17/11/23 16:58:45 INFO mapreduce.Job:  map 0% reduce 0%
17/11/23 16:58:51 INFO mapreduce.Job:  map 75% reduce 0%
17/11/23 16:58:55 INFO mapreduce.Job:  map 100% reduce 0%
17/11/23 16:58:57 INFO